In [2]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=4313e80645f3cb85d6d3b0a9b0ae59db64970b3adc9fa04f63ecde9cacde78c1
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


## Import the dataset
no X and Y,no scikitlearn,use **apyori** 
> pd.read_csv() always assumes the first row as column header and treats it as column names so it is necessary to mention if the data doesn't have a col header with the arg => 'header = None'



In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Market_Basket_Optimisation.csv", header = None)
print (len(df))

7501


##Data Preprocessing
Have to *recreate the dataset as apyori expects* a data to be a **list of transactions** - to create a list, we start with a empty[] and use for loop to populate it :)

In [14]:
transactions_list = []
for i in range(0, len(df)):
  transactions_list.append([str(df.values[i,j]) for j in range(0,20)])    ##we are creating the list of lists, we choose range(0,20)- i.e the len of the longest transaction

##Train the dataset

In [15]:
from apyori import apriori
rules = apriori(transactions = transactions_list, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2 ) ##our choice is min_sup -- no default value is prescribed, min_conf def val = 0.8
## no.of prod you want in lhs and rhs is decided by min_len and max_len arguments


In [16]:
results = list(rules)
print (results)

[RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]), RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]), RelationRecord(items=frozenset({'escalope', 'pasta'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]), RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0.24

In [18]:
##Put the results in dataframe for proper view
def inspect(results):
  lhs = [ tuple(res[2][0][0])[0] for res in results ]
  rhs = [ tuple(res[2][0][1])[0] for res in results ]
  supports = [ res[1] for res in results ] 
  confidences = [ res[2][0][2] for res in results ]
  lifts = [ res[2][0][3] for res in results ]
  return list(zip(lhs, rhs, supports, confidences, lifts))

resultsindataframe = pd.DataFrame(inspect(results), columns=['LHS', 'RHS', "Support", 'Confidence', 'Lift'])
resultsindataframe

,LHS,RHS,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


In [19]:
##to print in sorted order we use 'nlargest()'
resultsindataframe.nlargest(n = 10, columns = 'Lift')  ## 'n' how many rules you want(or num of rows you want to return), 'columns' based on which col you want to sort


,LHS,RHS,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710


you can make business decisions on these rules! :)